In [2]:
# --- COMPLETE SINGLE FILE CODE FOR GROQ FINANCIAL CHATBOT ANALYSIS ---

# --- SECTION 0: IMPORTS AND SETUP ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import ast
import re
import os

plt.style.use("ggplot")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

LOG_FILE = "chat_log.csv"
OUTPUT_CSV = "processed_chat_analysis.csv"

# Helper function to safely parse the profile string
def parse_profile(profile_str):
    """Safely converts the profile string representation of a dict to a dict."""
    if pd.isna(profile_str) or profile_str == '':
        return {}
    try:
        # Simple cleanup for dict-like string issues
        profile_str = profile_str.replace("nan", "None").replace("true", "True").replace("false", "False").strip()
        return ast.literal_eval(profile_str)
    except:
        return {}


# --- SECTION 1: LOAD AND CLEAN DATA ---
print("=== Loading and Cleaning Data ===")
columns = [
    "timestamp", "user_query", "llm_mode", "rag_context",
    "profile_used", "assistant_response", "latency_sec"
]

# Set header to None and skip bad lines
df = pd.read_csv(LOG_FILE, header=None, names=columns, on_bad_lines='skip')

# Convert data types and clean
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df["latency_sec"] = pd.to_numeric(df["latency_sec"], errors="coerce")
df.dropna(subset=["latency_sec", "llm_mode"], inplace=True)

# Calculate length features
df["query_length"] = df["user_query"].astype(str).apply(len)
df["response_length"] = df["assistant_response"].astype(str).apply(len)
df["context_length"] = df["rag_context"].astype(str).apply(len)

print(f"✅ Loaded and cleaned {len(df)} records.\n")
print("--- Data Head (First 3 Rows) ---")
print(df.head(3).to_markdown(floatfmt=".4f"))


# --- SECTION 2: BASIC STATS AND MODE DISTRIBUTION ---
print("\n=== Basic Statistics and Mode Distribution ===")
mode_counts = df["llm_mode"].value_counts()
print(f"Total Queries: {len(df)}")
print(f"Average Latency: {df['latency_sec'].mean():.4f} sec")
print("\nMode Distribution:")
print(mode_counts.to_markdown())

# Visualization: Mode Distribution
plt.figure(figsize=(6, 4))
sns.countplot(x="llm_mode", data=df, palette="coolwarm", order=mode_counts.index)
plt.title("Distribution of Chatbot Modes")
plt.xlabel("LLM Mode")
plt.ylabel("Number of Queries")
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig("mode_distribution.png")
plt.close()


# --- SECTION 3: CORE LATENCY COMPARISON ---
df_modes = df[df["llm_mode"].isin(["RAG (Education)", "Decision Support"])].copy()

latency_summary = df_modes.groupby("llm_mode")["latency_sec"].agg(["mean", "median", "std"])
print("\n=== Latency Statistics (RAG vs. Decision Support) ===")
print(latency_summary.to_markdown(floatfmt=".4f"))

# Visualization: Latency Boxplot
plt.figure(figsize=(7, 5))
sns.boxplot(x="llm_mode", y="latency_sec", data=df_modes, palette="pastel")
plt.title("Latency by Operational Mode")
plt.xlabel("LLM Mode")
plt.ylabel("Latency (seconds)")
plt.tight_layout()
plt.savefig("latency_boxplot.png")
plt.close()


# --- SECTION 4: THROUGHPUT ANALYSIS (Character/Second) ---
df_modes["char_per_sec"] = df_modes["response_length"] / df_modes["latency_sec"]
df_modes.replace([np.inf, -np.inf], np.nan, inplace=True) # Handle division by zero (latency=0)

throughput_summary = df_modes.groupby("llm_mode")["char_per_sec"].agg(["mean", "median", "std"]).dropna()
print("\n=== Throughput (Characters/Second) ===")
print(throughput_summary.to_markdown(floatfmt=".2f"))

# Visualization: Throughput Boxplot
plt.figure(figsize=(7, 5))
sns.boxplot(x="llm_mode", y="char_per_sec", data=df_modes.dropna(subset=['char_per_sec']), palette="cool")
plt.title("LLM Throughput (Characters/Second) by Mode")
plt.xlabel("LLM Mode")
plt.ylabel("Throughput (Char/Sec)")
plt.tight_layout()
plt.savefig("throughput_boxplot.png")
plt.close()


# --- SECTION 5: RAG CONTEXT OVERHEAD ---
df_rag = df[df['llm_mode'] == 'RAG (Education)'].copy()

# Statistical Test: Pearson Correlation for RAG mode only (Context Length vs. Latency)
valid_rag_data = df_rag.dropna(subset=['context_length', 'latency_sec'])
corr_rag, p_rag = stats.pearsonr(valid_rag_data['context_length'], valid_rag_data['latency_sec'])

print(f"\n=== RAG Context Length vs. Latency (Pearson Correlation) ===")
print(f"RAG Correlation Coefficient (r): {corr_rag:.4f}")
print(f"P-value: {p_rag:.4e}")
print(f"Conclusion: {'Reject Null (Significant linear correlation)' if p_rag < 0.05 else 'Fail to Reject Null'}")

# Visualization: Context Length vs. Latency
plt.figure(figsize=(8, 6))
plt.hexbin(df_rag['context_length'], df_rag['latency_sec'], gridsize=20, cmap='viridis')
cb = plt.colorbar(label='Count of Queries')
plt.title('RAG Context Length vs. Latency')
plt.xlabel('RAG Context Length (Characters)')
plt.ylabel('Latency (Seconds)')
plt.tight_layout()
plt.savefig("rag_context_latency_hexbin.png")
plt.close()


# --- SECTION 6: ADVANCED STATISTICAL TESTS ---
print("\n=== Advanced Statistical Tests ===")

# 1. T-Test: RAG vs. Decision Support Latency (Core research novelty)
df_rag_lat = df[df['llm_mode'] == 'RAG (Education)']['latency_sec'].dropna()
df_ds_lat = df[df['llm_mode'] == 'Decision Support']['latency_sec'].dropna()

t_stat, p_t_test = stats.ttest_ind(df_rag_lat, df_ds_lat, equal_var=False)

print(f"\n1. Two-Sample T-Test (RAG vs. Decision Support Latency):")
print(f"  RAG Mean Latency: {df_rag_lat.mean():.4f} sec")
print(f"  DS Mean Latency: {df_ds_lat.mean():.4f} sec")
print(f"  T-statistic: {t_stat:.4f}, P-value: {p_t_test:.4e}")
print(f"  Conclusion: {'Reject Null (Latency is significantly different)' if p_t_test < 0.05 else 'Fail to Reject Null'}")


# 2. ANOVA Test: Mean Latency across ALL Modes
unique_modes = df['llm_mode'].unique()
mode_groups = [df['latency_sec'][df['llm_mode'] == mode].dropna() for mode in unique_modes]
f_stat, p_anova = stats.f_oneway(*mode_groups)

print(f"\n2. ANOVA Test (Mean Latency across All Modes):")
print(f"  F-statistic: {f_stat:.4f}, P-value: {p_anova:.4e}")
print(f"  Conclusion: {'Reject Null (Significant difference in means)' if p_anova < 0.05 else 'Fail to Reject Null'}")


# --- SECTION 7: PERSONALIZATION IMPACT ANALYSIS (DECISION SUPPORT MODE) ---
df['profile_data'] = df['profile_used'].apply(parse_profile)

df_ds_parsed = df[df['llm_mode'] == 'Decision Support'].copy()
df_ds_parsed['Income'] = df_ds_parsed['profile_data'].apply(lambda x: x.get('Income'))
df_ds_parsed['Age'] = df_ds_parsed['profile_data'].apply(lambda x: x.get('Age'))
df_ds_parsed['Desired_Savings'] = df_ds_parsed['profile_data'].apply(lambda x: x.get('Desired_Savings'))

# Create income tiers
df_ds_parsed['Income_Tier'] = pd.cut(
    df_ds_parsed['Income'],
    bins=[0, 30000, 70000, np.inf],
    labels=['Low (<30k)', 'Medium (30k-70k)', 'High (>70k)'],
    right=False
)

# Use observed=True for categorical grouping
income_impact = df_ds_parsed.groupby('Income_Tier', observed=True)[['response_length', 'latency_sec']].agg(['mean', 'median', 'count']).dropna()

print("\n=== Income Tier Impact on Decision Support ===")
print(income_impact.to_markdown(floatfmt=".1f"))

# Visualization: Income Tier vs. Response Length
plt.figure(figsize=(7, 5))
sns.barplot(x='Income_Tier', y='response_length', data=df_ds_parsed, palette='viridis', errorbar=None)
plt.title('Mean Response Length by User Income Tier (Decision Support)')
plt.xlabel('Income Tier')
plt.ylabel('Mean Response Length (Characters)')
plt.tight_layout()
plt.savefig("income_response_length_bar.png")
plt.close()


# --- SECTION 8: CORRELATION HEATMAP ---
corr_cols = [
    'latency_sec', 'response_length', 'query_length', 'context_length', 'Income', 'Age', 'Desired_Savings'
]
# Ensure that we only use rows with full data for the correlation
df_corr = df_ds_parsed[corr_cols].dropna()

corr_matrix = df_corr.corr()

# Visualization: Correlation Heatmap
plt.figure(figsize=(9, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, cbar_kws={'label': 'Pearson Correlation Coefficient'})
plt.title('Correlation Matrix of Performance and Profile Metrics (Decision Support)')
plt.tight_layout()
plt.savefig("correlation_heatmap.png")
plt.close()


# --- SECTION 9: EXPORT CLEAN DATA ---
# Drop the intermediate 'profile_data' column before export
df_export = df.drop(columns=['profile_data'])
df_export.to_csv(OUTPUT_CSV, index=False)
print(f"\n✅ Cleaned dataset saved to: {OUTPUT_CSV}")

=== Loading and Cleaning Data ===
✅ Loaded and cleaned 4056 records.

--- Data Head (First 3 Rows) ---
|    | timestamp                  | user_query                                      | llm_mode        | rag_context                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | profile_used                                                                                                                                                                                                                                                                   

/tmp/ipython-input-3918674890.py:68: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x="llm_mode", data=df, palette="coolwarm", order=mode_counts.index)



=== Latency Statistics (RAG vs. Decision Support) ===
| llm_mode         |   mean |   median |    std |
|:-----------------|-------:|---------:|-------:|
| Decision Support | 4.4231 |   2.5920 | 6.8951 |
| RAG (Education)  | 2.5433 |   2.7599 | 0.8416 |


/tmp/ipython-input-3918674890.py:87: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x="llm_mode", y="latency_sec", data=df_modes, palette="pastel")



=== Throughput (Characters/Second) ===
| llm_mode         |    mean |   median |    std |
|:-----------------|--------:|---------:|-------:|
| Decision Support |  917.61 |   559.01 | 879.37 |
| RAG (Education)  | 1011.21 |   777.12 | 735.87 |

=== RAG Context Length vs. Latency (Pearson Correlation) ===
RAG Correlation Coefficient (r): 0.1531
P-value: 3.8351e-07
Conclusion: Reject Null (Significant linear correlation)


/tmp/ipython-input-3918674890.py:106: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x="llm_mode", y="char_per_sec", data=df_modes.dropna(subset=['char_per_sec']), palette="cool")



=== Advanced Statistical Tests ===

1. Two-Sample T-Test (RAG vs. Decision Support Latency):
  RAG Mean Latency: 2.5433 sec
  DS Mean Latency: 4.4231 sec
  T-statistic: -8.8734, P-value: 2.8004e-18
  Conclusion: Reject Null (Latency is significantly different)

2. ANOVA Test (Mean Latency across All Modes):
  F-statistic: 551.8683, P-value: 1.0751e-212
  Conclusion: Reject Null (Significant difference in means)

=== Income Tier Impact on Decision Support ===
| Income_Tier      |   ('response_length', 'mean') |   ('response_length', 'median') |   ('response_length', 'count') |   ('latency_sec', 'mean') |   ('latency_sec', 'median') |   ('latency_sec', 'count') |
|:-----------------|------------------------------:|--------------------------------:|-------------------------------:|--------------------------:|----------------------------:|---------------------------:|
| Low (<30k)       |                        1457.1 |                          1417.0 |                          535.0 |   

/tmp/ipython-input-3918674890.py:189: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Income_Tier', y='response_length', data=df_ds_parsed, palette='viridis', errorbar=None)



✅ Cleaned dataset saved to: processed_chat_analysis.csv
